In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.datasets import mnist
import tensorflow as tf
from keras.backend import map_fn
from keras.layers import Input, Dense, Lambda, concatenate, Add
from keras.models import Model
from keras import backend as K
from keras.engine.topology import Layer
from keras.backend import exp, log
from sklearn.preprocessing import MinMaxScaler

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
x_train.shape

In [ ]:
noisy_input_data = np.zeros((len(x_train), 784))

In [ ]:
for i in range(len(x_train)):
    noise = np.random.normal(loc=0.0, scale=(1.0/256.0), size=(28,28))
    noisy_data = x_train[i] + noise
    noisy_data = noisy_data.reshape((784,))
    noisy_input_data[i] = noisy_data

In [ ]:
noisy_input_data.shape

In [ ]:
maxmin = MinMaxScaler()
noisy_input_data = maxmin.fit_transform(noisy_input_data)
noisy_input_data = (noisy_input_data > 0.5).astype(np.float32)

In [ ]:
def create_network(input_shape=392):
    a = Input(shape=(input_shape,))
    b = Dense(1000, activation='relu')(a)
    c = Dense(1000, activation='relu')(b)
    d = Dense(1000, activation='relu')(c)
    e = Dense(1000, activation='relu')(d)
    f = Dense(1000, activation='relu')(e)
    g = Dense(392)(f)
    model = Model(inputs=a, outputs=g)
    return model
    

In [ ]:
class ScalingLayer(Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(ScalingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(input_shape[1],),
                                      initializer='uniform',
                                      trainable=True)
        super(ScalingLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        ss = tf.expand_dims(self.kernel, 0)
        dd = tf.multiply(x, exp(self.kernel))
        #print(dd.shape)
        return dd

    def compute_output_shape(self, input_shape):
        return input_shape

In [ ]:
x_odd = Input(batch_shape=(None,392))
x_even = Input(batch_shape=(None,392))

nn_1 = create_network()
h1_odd = x_odd
a1 = nn_1(x_odd)
h1_even = Add()([a1, x_even])

nn_2 = create_network()
a2 = nn_2(x_even)
h2_odd = Add()([h1_odd, a2])
h2_even = h1_even

nn_3 = create_network()
h3_odd = h2_odd
a3 = nn_3(h1_odd)
h3_even = Add()([h2_odd, a3])

nn_4 = create_network()
a4 = nn_4(x_even)
h4_odd = Add()([h3_odd, a4])
h4_even = h3_even

h = concatenate([h4_even, h4_odd])

scaling_layer = ScalingLayer(output_dim=h.shape)
final_h = scaling_layer(h)

final_model = Model(inputs=(x_odd, x_even), outputs=final_h)

In [ ]:
final_model.summary()

In [ ]:
def logistic_distribution(hd):
    logprob = log(1.0+exp(hd)) + log(1.0+exp(-1*hd))
    logprob = -1.0 * logprob
    return logprob

In [ ]:
def log_loss(ss):
  def main_loss(y_true, y_pred):
    total_ss = K.sum(ss)
    total_hh = K.sum(map_fn(logistic_distribution, y_pred))
    return total_ss + total_hh
  return main_loss


In [ ]:
from keras.optimizers import Adam, RMSprop
opt=Adam(lr=0.001)

In [ ]:
final_model.compile(optimizer=opt,loss=log_loss(final_model.layers[-1].trainable_weights))

In [ ]:
x_even = noisy_input_data[:, 1::2] #even
x_odd = noisy_input_data[:, 0::2] #odd

In [ ]:
final_model.fit(x=[x_even, x_odd], y=x_even, steps_per_epoch=1000)